# Processing California's public payrolls

This notebook processess government payroll [data](https://publicpay.ca.gov/Reports/RawExport.aspx) compiled and released annually by the California state controller's office. The data include anonymized salary information for all employees at cities, counties, special districts and state government. 

---

### Load python tools

In [1]:
import pandas as pd
import zipfile
from urllib.request import urlopen 
import pyarrow
import os
import cpi
import glob
import requests
import matplotlib
import json
import numpy as np
from altair import datum
import altair as alt
import altair_latimes as lat
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')
pd.options.display.float_format = '{:,.0f}'.format

### Download zipped salary tables by year and agency type

In [2]:
cpi.update()

In [3]:
os.chdir('/Users/mhustiles/data/data/controller/input/')

In [4]:
formaturl = lambda x: 'https://publicpay.ca.gov/RawExport/' + f'{x[1]}_' + f'{x[0]}' + '.zip'

In [5]:
metadata = []
for y in range(2009,2020):
    for e in ['City', 'County', 'SpecialDistrict', 'StateDepartment']:
#     for e in ['City', 'County']:
        metadata.append(dict(entity = e, year = y, url = formaturl((e, y))))

### Extract CSVs from .zip files, and then discard the .zip files

In [6]:
for m in metadata:
    !wget '{m['url']}'
    !unzip \*.zip
    !rm -f *.zip

--2020-10-12 13:04:25--  https://publicpay.ca.gov/RawExport/2009_City.zip
Resolving publicpay.ca.gov (publicpay.ca.gov)... 67.157.158.198
Connecting to publicpay.ca.gov (publicpay.ca.gov)|67.157.158.198|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5052183 (4.8M) [application/x-zip-compressed]
Saving to: ‘2009_City.zip’

2009_City.zip       100%[===================>]   4.82M  1.65MB/s    in 2.9s    

2020-10-12 13:04:29 (1.65 MB/s) - ‘2009_City.zip’ saved [5052183/5052183]

Archive:  2009_City.zip
  inflating: 2009_City.csv           
--2020-10-12 13:04:29--  https://publicpay.ca.gov/RawExport/2009_County.zip
Resolving publicpay.ca.gov (publicpay.ca.gov)... 67.157.158.198
Connecting to publicpay.ca.gov (publicpay.ca.gov)|67.157.158.198|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5856004 (5.6M) [application/x-zip-compressed]
Saving to: ‘2009_County.zip’

2009_County.zip     100%[===================>]   5.58M  1.73MB/s    in 3.3s 

---

### Read all the text files, loop and store them in a dataframe

In [7]:
path = '/Users/mhustiles/data/data/controller/input/'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_from_each_file = (pd.read_csv(f, encoding = "ISO-8859-1", \
            low_memory=False, dtype = {'DepartmentOrSubdivision': 'object', 'Year': 'object'}) for f in all_files)
concatenated_df = pd.concat(df_from_each_file, ignore_index=True)

### Trim the dataframe to the columns we need

In [8]:
payroll = \
pd.DataFrame(concatenated_df[['Year','EmployerType','EmployerPopulation','EmployerName','DepartmentOrSubdivision',\
                 'Position','OvertimePay','TotalWages', 'TotalRetirementAndHealthContribution', 'EmployerCounty']])

### Not everyone reports (or pays) overtime

In [9]:
payroll['OvertimePay'].fillna(0, inplace = True)
payroll['DepartmentOrSubdivision'].fillna('NOT LISTED', inplace = True)
payroll['EmployerPopulation'].fillna(0, inplace = True)

### Clean up column headers

In [10]:
payroll.columns = payroll.columns.str.strip().str.lower().str.replace(' ', '_')\
                    .str.replace('(', '').str.replace(')', '').str.replace('-','_')

In [11]:
payroll.rename(columns = {
'employertype':'type',
'employerpopulation':'population',
'employername':'employer',
'departmentorsubdivision':'department',
'overtimepay':'overtime',
'totalretirementandhealthcontribution':'benefits',
'totalwages':'wages',
 }, inplace = True)

### Uppercase everything because their title casing across hundreds of agencies is janky

In [12]:
payroll = payroll.apply(lambda x: x.astype(str).str.upper())

### Convert year column to datetime year for CPI function

In [13]:
payroll['year'] = pd.to_datetime(payroll['year'])

In [14]:
payroll['year'] = payroll['year'].dt.year

### Adjust overtime on each row for CPI according to its respective year

In [15]:
payroll[['overtime', 'population', 'wages', 'benefits']] = \
payroll[['overtime', 'population', 'wages', 'benefits']].astype(float)

In [16]:
payroll['basewages'] = (payroll['wages'] - payroll['overtime']).astype(float)

In [17]:
payroll['adjusted_overtime'] = payroll.apply(lambda x: cpi.inflate(x.overtime, x.year), axis=1)

In [18]:
payroll['adjusted_wages'] = payroll.apply(lambda x: cpi.inflate(x.wages, x.year), axis=1)

In [19]:
payroll['adjusted_benefits'] = payroll.apply(lambda x: cpi.inflate(x.benefits, x.year), axis=1)

In [20]:
payroll['adjusted_basewages'] = payroll.apply(lambda x: cpi.inflate(x.basewages, x.year), axis=1)

### How does the dataframe look? 

In [21]:
payroll.head()

,year,type,population,employer,department,position,overtime,wages,benefits,employercounty,basewages,adjusted_overtime,adjusted_wages,adjusted_benefits,adjusted_basewages
0,2010,COUNTY,"1,517,756",ALAMEDA,AC CENTRAL COLLECTIONS,ACCOUNTANT-AUDITOR,0,"4,120","1,088",ALAMEDA,"4,120",0,"4,830","1,276","4,830"
1,2010,COUNTY,"1,517,756",ALAMEDA,AC CENTRAL COLLECTIONS,ACCOUNTING TECHNICIAN,0,"54,438","14,031",ALAMEDA,"54,438",0,"63,825","16,450","63,825"
2,2010,COUNTY,"1,517,756",ALAMEDA,AC CENTRAL COLLECTIONS,ADMIN SPECIALIST II,0,"25,376","5,486",ALAMEDA,"25,376",0,"29,752","6,432","29,752"
3,2010,COUNTY,"1,517,756",ALAMEDA,AC CENTRAL COLLECTIONS,AUDITOR ASSOCIATE I,0,"31,225","8,710",ALAMEDA,"31,225",0,"36,609","10,212","36,609"
4,2010,COUNTY,"1,517,756",ALAMEDA,AC CENTRAL COLLECTIONS,AUDITOR ASSOCIATE I,0,"27,677","4,916",ALAMEDA,"27,677",0,"32,450","5,764","32,450"


### How many records do we have here?

In [22]:
# How many records?
len(payroll)

11984248

### Clean up some employer names

In [23]:
payroll['employer']\
    .replace('SACRAMENTO METROPOLITAN FIRE DISTRICT', 'SACRAMENTO METRO FIRE', inplace=True)

In [24]:
payroll['employer']\
    .replace('FORESTRY & FIRE PROTECTION, CALIFORNIA DEPARTMENT OF', 'CALFIRE', inplace=True)

In [25]:
payroll['type']\
    .replace('STATE DEPARTMENT', 'STATE', inplace=True)

In [26]:
payroll['employer']\
    .replace('ORANGE COUNTY FIRE AUTHORITY', 'OC FIRE AUTHORITY', inplace=True)

### Export table of all titles to help ID fire roles

In [27]:
positions_all = payroll.groupby(['position', 'employer', 'type', 'department'])\
    .agg('size').reset_index()

In [28]:
positions_all['position'] = positions_all['position']\
                            .str.replace('!ST ', '', regex=False)\
                            .str.replace('1ST ', '', regex=False)\
                            .str.replace('2ND', '', regex=False)\
                            .str.replace('.E.', '', regex=False)\
                            .str.replace(' - ', '', regex=False)\
                            .str.replace('\d', '')\
                            .str.replace('[.!?\\-]', '')

In [29]:
positions_fire = positions_all[ (positions_all['position'].str.contains('FIRE')) |\
                                       (positions_all['type'].str.contains('FIRE')) |\
                                       (positions_all['department'].str.contains('FIRE')) |\
                                       (positions_all['employer'].str.contains('FIRE'))]

In [30]:
positions_fire_grouped = positions_fire.groupby(['position']).agg('size')

In [32]:
positions_fire_grouped.to_csv('/Users/mhustiles/data/github/notebooks/california-fire-payroll/input/positions_fire_grouped.csv', header=True)

---

### Export to a lightweight format

In [33]:
payroll.reset_index().to_feather('/Users/mhustiles/data/data/controller/output/payroll.feather')

In [34]:
payroll.reset_index().to_csv('/Users/mhustiles/data/data/controller/output/payroll.csv')

---

Data source: https://publicpay.ca.gov/Reports/RawExport.aspx